# Daily Safe Strategy

In [295]:
import pandas as pd
df = pd.read_csv("EURUSD_Candlestick_1_D_ASK_05.05.2003-12.02.2022.csv")
#df = pd.read_csv("EURUSD_Candlestick_4_Hour_ASK_05.05.2003-12.02.2022.csv")
#df = pd.read_csv("AMZN.USUSD_Candlestick_1_D_ASK_26.01.2017-12.02.2022.csv")
#df = pd.read_csv("EBAY.USUSD_Candlestick_1_D_ASK_02.11.2017-12.02.2022.csv")
#df = pd.read_csv("VOW3.DEEUR_Candlestick_1_D_ASK_14.04.2015-12.02.2022.csv")
df.tail()

,Local time,Open,High,Low,Close,Volume
6854,08.02.2022 00:00:00.000 GMT+0200,1.14431,1.14489,1.13963,1.14174,76392.8472
6855,09.02.2022 00:00:00.000 GMT+0200,1.14175,1.14480,1.14027,1.14261,63268.7700
6856,10.02.2022 00:00:00.000 GMT+0200,1.14263,1.14951,1.13750,1.14292,112513.0050
6857,11.02.2022 00:00:00.000 GMT+0200,1.14289,1.14331,1.13299,1.13507,129641.5800
6858,12.02.2022 00:00:00.000 GMT+0200,1.13507,1.13507,1.13507,1.13507,0.0000


In [296]:
#Check if NA values are in data
df.columns=['time', 'Open', 'High', 'Low', 'Close', 'Volume']
df=df[df['Volume']!=0]
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df.tail()

,time,Open,High,Low,Close,Volume
4891,07.02.2022 00:00:00.000 GMT+0200,1.14677,1.14725,1.14154,1.14431,90235.8900
4892,08.02.2022 00:00:00.000 GMT+0200,1.14431,1.14489,1.13963,1.14174,76392.8472
4893,09.02.2022 00:00:00.000 GMT+0200,1.14175,1.14480,1.14027,1.14261,63268.7700
4894,10.02.2022 00:00:00.000 GMT+0200,1.14263,1.14951,1.13750,1.14292,112513.0050
4895,11.02.2022 00:00:00.000 GMT+0200,1.14289,1.14331,1.13299,1.13507,129641.5800


In [297]:
import pandas_ta as ta
df["EMA"] = ta.ema(df.Close, length=200)
df['ATR']= df.ta.atr()

# EMA Signal

In [298]:
emasignal = [0]*len(df)
backcandles = 4

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if df.High[i]>=df.EMA[i]:
            dnt=0
        if df.Low[i]<=df.EMA[i]:
            upt=0
    if upt==1 and dnt==1:
        #print("!!!!! check trend loop !!!!")
        emasignal[row]=3
    elif upt==1:
        emasignal[row]=2
    elif dnt==1:
        emasignal[row]=1

df['EMASignal'] = emasignal

# Volume Signal

In [299]:
VSignal = [0] * len(df)
vbackcandles = 1
for row in range(vbackcandles+1, len(df)):
    VSignal[row] = 1
    for i in range(row-vbackcandles, row):
        if df.Volume[row]<df.Volume[i] and df.Volume[row-1]<df.Volume[row-2]:
            VSignal[row]=0
df['VSignal']=VSignal

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

# Price Signal

In [300]:
PriceSignal = [0]*len(df)
pbackcandles = 4
for row in range(pbackcandles, len(df)):
    PriceSignal[row] = 1
    for i in range(row-pbackcandles, row):
        if df.EMASignal[row] == 1: #downtrend
            if df.Open[row]<=df.Close[row]: #downcandle row
                PriceSignal[row]=0
            elif df.Open[i]>df.Close[i]: #downcandle i we are looking for 4 upcandles
                PriceSignal[row]=0
        if df.EMASignal[row] == 2: #uptrend
            if df.Open[row]>=df.Close[row]: #upcandle row
                PriceSignal[row]=0
            elif df.Open[i]<df.Close[i]: #upcandle i we are looking for 4 dowcandles
                PriceSignal[row]=0
        else:
            PriceSignal[row] = 0

df['PriceSignal']=PriceSignal

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

# Total Signal

In [301]:
TotSignal = [0] * len(df)
for row in range(0, len(df)):
    if df.EMASignal[row]==1 and df.VSignal[row]==1 and df.PriceSignal[row]==1:
        TotSignal[row]=1
    if df.EMASignal[row]==2 and df.VSignal[row]==1 and df.PriceSignal[row]==1:
        TotSignal[row]=2

df['TotSignal']=TotSignal

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

# Visualizing signals on chart

In [302]:
import numpy as np
def pointpos(x):
    if x['TotSignal']==1:
        return x['High']+2e-3
    elif x['TotSignal']==2:
        return x['Low']-2e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [303]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

dfpl = df[570:900]

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.EMA, line=dict(color='red', width=1), name="EMA")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=8, color="MediumPurple"),
                name="Signal")
fig.show()

In [315]:
df.iloc[848:851]
#df[df["TotSignal"]==2]

,time,Open,High,Low,Close,Volume,EMA,ATR,EMASignal,VSignal,PriceSignal,TotSignal,pointpos,SLSignal
848,09.05.2007 00:00:00.000 GMT+0300,1.35437,1.35652,1.35199,1.35278,505196.9004,1.307173,0.006926,2,1,0,0,NaN,1.35170
849,10.05.2007 00:00:00.000 GMT+0300,1.35273,1.35635,1.34666,1.34851,507636.7042,1.307584,0.007123,2,1,0,0,NaN,1.34666
850,11.05.2007 00:00:00.000 GMT+0300,1.34845,1.35317,1.34637,1.35238,507333.7011,1.308030,0.007100,2,1,1,2,1.34437,1.34637


# Stop Loss from Price

In [305]:
SLSignal = [0] * len(df)
SLbackcandles = 4
for row in range(SLbackcandles, len(df)):
    mi=1e10
    ma=-1e10
    if df.EMASignal[row]==1:
        for i in range(row-SLbackcandles, row+1):
            ma = max(ma,df.High[i])
        SLSignal[row]=ma
    if df.EMASignal[row]==2:
        for i in range(row-SLbackcandles, row+1):
            mi = min(mi,df.Low[i])
        SLSignal[row]=mi
        
df['SLSignal']=SLSignal

# Backtesting Strategy

In [306]:
dfpl = df[:]
def SIGNAL():
    return dfpl.TotSignal

In [307]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.2
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        slatr = 4*self.data.ATR[-1]
        TPSLRatio = 1.5

        if self.signal1==2 and len(self.trades)==0:   
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=10000, margin=1/50, commission=.00)
stat = bt.run()
stat

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                    4696.0
Duration                               4696.0
Exposure Time [%]                   22.099212
Equity Final [$]                 29436.742713
Equity Peak [$]                  40628.190691
Return [%]                         194.367427
Buy & Hold Return [%]              -10.642703
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -61.52906
Avg. Drawdown [%]                  -11.912958
Max. Drawdown Duration                 1759.0
Avg. Drawdown Duration             132.033333
# Trades                                 13.0
Win Rate [%]                        61.538462
Best Trade [%]                       6.007887
Worst Trade [%]                     -3.819902
Avg. Trade [%]                    

In [308]:
bt.plot(show_legend=False)

Row(id='34085', ...)

In [309]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.2
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 1.5

        if self.signal1==2 and len(self.trades)==0:   
            sl1 = self.data.SLSignal[-1]
            tp1 = self.data.Close[-1]+(self.data.Close[-1] - sl1)*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.SLSignal[-1]
            tp1 = self.data.Close[-1]-(sl1 - self.data.Close[-1])*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=10000, margin=1/50, commission=.00)
stat = bt.run()
stat

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                    4696.0
Duration                               4696.0
Exposure Time [%]                    2.810304
Equity Final [$]                   19010.7147
Equity Peak [$]                    19010.7147
Return [%]                          90.107147
Buy & Hold Return [%]              -10.642703
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -22.218899
Avg. Drawdown [%]                   -9.256062
Max. Drawdown Duration                  974.0
Avg. Drawdown Duration             259.818182
# Trades                                 23.0
Win Rate [%]                        56.521739
Best Trade [%]                       2.722009
Worst Trade [%]                     -1.061614
Avg. Trade [%]                    

In [310]:
bt.plot(show_legend=False)

Row(id='34871', ...)